## Лаба 3. Рекомендательная система видеоконтента с implicit feedback – Spark ML

#### Дедлайн

Понедельник, 15 марта, 23:59

#### Дедлайн Github

Четверг, 18 марта, 23:59

#### Задача

В вашем распоряжении имеется уже предобработанный и очищенный датасет с фактами покупок абонентами телепередач от компании E-Contenta. По доступным вам данным нужно предсказать вероятность покупки других передач этими, а, возможно, и другими абонентами. 
При решении задачи запрещено использовать библиотеки pandas, sklearn (кроме sklearn.metrics), xgboost и другие. 
Если scikit-learn (например, но и другие тоже) обернут в классы Transformer и Estimator, то их можно использовать.

#### Описание данных

Для выполнения работы вам следует взять все файлы из папки на HDFS `/labs/slaba03/`. 

Давайте посмотрим, что у нас есть:

```
$ hdfs dfs -ls /labs/slaba03/
Found 4 items
-rw-r--r--   3 hdfs hdfs   91066524 2019-03-17 21:07 /labs/slaba03/laba03_items.csv
-rw-r--r--   3 hdfs hdfs   29965581 2019-03-17 21:07 /labs/slaba03/laba03_test.csv
-rw-r--r--   3 hdfs hdfs   74949368 2019-03-17 21:07 /labs/slaba03/laba03_train.csv
-rw-r--r--   3 hdfs hdfs  871302535 2019-03-17 21:07 /labs/slaba03/laba03_views_programmes.csv
```

- В `laba03_train.csv` содержатся факты покупки (колонка `purchase`) пользователями (колонка `user_id`) телепередач (колонка `item_id`). Такой формат файла вам уже знаком.

- `laba03_items.csv` — дополнительные данные по items. В данном файле много лишней или ненужной информации, так что задача её фильтрации и отбора ложится на вас. Поля в файле, на которых хотелось бы остановиться:

  - `item_id` — primary key. Соответствует `item_id` в предыдущем файле.
  - `content_type` — тип телепередачи (`1` — платная, `0` — бесплатная). Вас интересуют платные передачи.
  - `title` — название передачи, текстовое поле.
  - `year` — год выпуска передачи, число.
  - `genres` — поле с жанрами передачи, разделёнными через запятую.

- `laba03_test.csv` — тестовый датасет без указанного целевого признака `purchase`, который вам и предстоит предсказать.

- Дополнительный файл `laba03_views_programmes.csv` по просмотрам передач с полями:

  - `ts_start` — время начала просмотра.

  - `ts_end` — время окончания просмотра.

  - `item_type` — тип просматриваемого контента:

    - `live` — просмотр "вживую", в момент показа контента в эфире.
    - `pvr` — просмотр в записи, после показа контента в эфире.

#### Результат

Предсказание целевой переменной "купит/не купит" — хорошо знакомая вам задача бинарной классификации. Поскольку нам важны именно вероятности отнесения пары `(пользователь, товар)` к классу "купит" (`1`), то, на самом деле, вы можете подойти к проблеме с разных сторон:

1. Как к разработке рекомендательной системы: рекомендовать пользователю `user_id` топ-N лучших телепередач, которые были найдены по методике user-user / item-item коллаборативной фильтрации.
2. Как к задаче факторизации матриц: алгоритмы SVD, ALS, FM/FFM.
3. Как просто к задаче бинарной классификации. У вас есть два датасета, которые можно каким-то образом объединить, дополнительно обработать и сделать предсказания классификаторами (Spark ML).
4. Как к задаче регрессии. Поскольку от вас требуется предсказать не факт покупки, а его *вероятность*, то можно перевести задачу в регрессионную и решать её соответствующим образом.

#### Советы

1. На качество прогноза в большей степени влияет _качество признаков_, которые вы сможете придумать из имеющихся данных, нежели выбор и _сложность алгоритма_.
2. Качество входных данных также имеет сильное значени. Существует фраза "garbage in – garbage out". Мусор на входе – мусор на выходе. Потратьте время на подготовку и предобработку данных.
Путь к успеху в третьей лабораторной:
1. Сосредоточьтесь на формировании следующих фичей: по файлу 
laba03_train.csv сформируйте признаки, характеризирующие как интенсивно
покупает пользователь и "покупаемость" item'ов
2. возьмите достаточно мощную модель (например GBTRegressor из pyspark'а)

#### Проверка

Эта лаба проходит в формате соревнования. Для вас оно начинается, когда вы успешно пройдёте минимальный порог  — **AUC должен составить не менее 0.79**. После этого вы увидите лидерборд и сможете следить за результатами других участников.

Как уже было сказано, мы будем оценивать ваш алгоритм по метрике ROC AUC. Чекеру требуются *вероятности* в диапазоне `[0.0, 1.0]` отнесения пары `(пользователь, товар)` в тестовой выборке к классу "1" (купит).

**Важно!** Для точной проверки не забудьте отсортировать полученный файл по возрастанию идентификаторов пользователей (`user_id`), а затем — по возрастанию идентификаторов передач (`item_id`).
```
,user_id,item_id,purchase
0,1654,336,0.021805684684958027
1,1654,678,0.021805684684958027
2,1654,691,0.021805684684958027
3,1654,696,0.021805684684958027
...
```



Результат следует сохранить в файл `lab03.csv` в своей домашней директории.

Проверка осуществляется автоматическим скриптом из [Личного кабинета](http://lk-spark.newprolab.com/lab/slaba03).

Обязательное условие зачета лабораторной работы – это выкладка после дедлайна лабы своего решения в репозиторий через pull-request. Как это сделать, можно прочитать [здесь](/git.md). Если будут вопросы – спрашивайте в Slack.

# Решение

In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set("spark.app.name", "natasha pritykovskaya Spark RDD app") 

spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [3]:
!hdfs dfs -ls /labs/slaba03/

Found 4 items
-rw-r--r--   3 hdfs hdfs   91066524 2021-02-27 22:12 /labs/slaba03/laba03_items.csv
-rw-r--r--   3 hdfs hdfs   29965581 2021-02-27 22:12 /labs/slaba03/laba03_test.csv
-rw-r--r--   3 hdfs hdfs   74949368 2021-02-27 22:12 /labs/slaba03/laba03_train.csv
-rw-r--r--   3 hdfs hdfs  871302535 2021-02-27 22:12 /labs/slaba03/laba03_views_programmes.csv


In [4]:
import pickle
import pandas as pd
import numpy as np
import pyspark.sql.functions as f
from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import StructType, StructField, IntegerType, StringType 
from pyspark.sql.types import TimestampType, LongType, FloatType, ArrayType
from pyspark.ml import Estimator
from pyspark.ml.linalg import VectorUDT

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

from sklearn.linear_model import LogisticRegression
from pyspark import keyword_only
from pyspark.ml import Model, Estimator
from pyspark.ml.param import Param, Params, TypeConverters
from pyspark.ml.param.shared import HasFeaturesCol, HasLabelCol, HasPredictionCol

In [5]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier, RandomForestClassifier, LogisticRegression
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

## laba03_train

In [48]:
schema = StructType(fields=[
    StructField("user_id", IntegerType()), # cодержатся факты покупки
    StructField("item_id", IntegerType()), # пользователями
    StructField("purchase", FloatType()) #телепередач
])
df_train = spark.read.csv("/labs/slaba03/laba03_train.csv", header=True, schema=schema).cache() 
df_train.show(2,False, True)

-RECORD 0---------
 user_id  | 1654  
 item_id  | 74107 
 purchase | 0.0   
-RECORD 1---------
 user_id  | 1654  
 item_id  | 89249 
 purchase | 0.0   
only showing top 2 rows



In [40]:
#df_train.summary().show()

## laba03_test

In [47]:
schema = StructType(fields=[
    StructField("user_id", IntegerType()), # cодержатся факты покупки
    StructField("item_id", IntegerType()), # пользователями
    StructField("purchase", FloatType()) #телепередач
])
df_test = spark.read.csv("/labs/slaba03/laba03_test.csv", header=True, schema=schema).cache()  
df_test.show(2,False, True)

-RECORD 0---------
 user_id  | 1654  
 item_id  | 94814 
 purchase | null  
-RECORD 1---------
 user_id  | 1654  
 item_id  | 93629 
 purchase | null  
only showing top 2 rows



In [42]:
# df_test.summary().show()

+-------+------------------+-----------------+--------+
|summary|           user_id|          item_id|purchase|
+-------+------------------+-----------------+--------+
|  count|           2156840|          2156840|       0|
|   mean| 869652.3733920922|66896.00283609354|    null|
| stddev|60706.516163349734|35227.83130704647|    null|
|    min|              1654|              326|    null|
|    25%|            846164|            65668|    null|
|    50%|            885247|            79856|    null|
|    75%|            908588|            93606|    null|
|    max|            941450|           104165|    null|
+-------+------------------+-----------------+--------+



## laba03_items

In [10]:
schema = StructType(fields=[
    StructField("item_id", IntegerType()), # primary key, Соответствует item_id в предыдущем файле
    StructField("channel_id", FloatType()),
    StructField("datetime_availability_start", TimestampType()),
    StructField("datetime_availability_stop", TimestampType()),
    StructField("datetime_show_start", TimestampType()),
    StructField("datetime_show_stop", TimestampType()),
    StructField("content_type", IntegerType()), # — тип телепередачи (1 — платная, 0 — бесплатная).
    StructField("title", StringType()), # — название передачи, текстовое поле
    StructField("year", FloatType()), # — год выпуска передачи, число
    StructField("genres", StringType()),# genres — поле с жанрами передачи, разделёнными через запятую.
    StructField("region_id", FloatType())])

df_items = spark.read.csv("/labs/slaba03/laba03_items.csv", header=True, sep="\t", schema=schema).cache()  
df_items.show(2,False, True)

-RECORD 0-------------------------------------------------------------------------------
 item_id                     | 65667                                                    
 channel_id                  | null                                                     
 datetime_availability_start | 1970-01-01 03:00:00                                      
 datetime_availability_stop  | 2018-01-01 03:00:00                                      
 datetime_show_start         | null                                                     
 datetime_show_stop          | null                                                     
 content_type                | 1                                                        
 title                       | на пробах только девушки (all girl auditions)            
 year                        | 2013.0                                                   
 genres                      | Эротика                                                  
 region_id           

In [11]:
df_items.summary().show(truncate=False, vertical=True)

-RECORD 0----------------------------------------------
 summary      | count                                  
 item_id      | 635568                                 
 channel_id   | 631864                                 
 content_type | 635568                                 
 title        | 635568                                 
 year         | 3700                                   
 genres       | 635535                                 
 region_id    | 273304                                 
-RECORD 1----------------------------------------------
 summary      | mean                                   
 item_id      | 6791742.332593838                      
 channel_id   | 1787.092608536014                      
 content_type | 0.005827857916068776                   
 title        | 675.2237762237762                      
 year         | 2005.0624324324324                     
 genres       | null                                   
 region_id    | 28.877857623744987              

## laba03_views_programmes.csv

In [12]:
schema = StructType(fields=[
    StructField("user_id", IntegerType()),
    StructField("item_id", IntegerType()),
    StructField("ts_start", IntegerType()), # время начала просмотра.
    StructField("ts_end", StringType()), # время окончания просмотра.
    StructField("item_type", StringType()) 
    # тип просматриваемого контента: # live — просмотр "вживую", в момент показа контента в эфире. 
                                  # pvr — просмотр в записи, после показа контента в эфире.
])

df_views = spark.read.csv("/labs/slaba03/laba03_views_programmes.csv", header=True, schema=schema).cache()   
df_views.show(2,False, True)

-RECORD 0---------------
 user_id   | 0          
 item_id   | 7101053    
 ts_start  | 1491409931 
 ts_end    | 1491411600 
 item_type | live       
-RECORD 1---------------
 user_id   | 0          
 item_id   | 7101054    
 ts_start  | 1491412481 
 ts_end    | 1491451571 
 item_type | live       
only showing top 2 rows



In [13]:
#df_views.summary().show(truncate=False, vertical=True)

## Random solution
Важно! Для точной проверки не забудьте отсортировать полученный файл по возрастанию идентификаторов пользователей (user_id), а затем — по возрастанию идентификаторов передач (item_id).

In [14]:
def torand_func(a: pd.Series) -> pd.Series:
    return pd.Series(np.random.uniform(0,1,a.shape[0]))
torand = pandas_udf(torand_func, returnType=FloatType())

df_test = df_test.withColumn('purchase',torand(df_test['purchase']))

df_test.toPandas()[["user_id","item_id","purchase"]].sort_values(["user_id","item_id"]).to_csv('../lab03.csv', sep=',')

# Solution

In [49]:
def ith_(v, i):
    try:
        return float(v[i])
    except ValueError:
        return None

ith = f.udf(ith_, FloatType())

In [50]:
# content_type
df_train=df_train.join(df_items[["item_id","content_type", "year", "genres"]], ["item_id"])
df_test=df_test.join(df_items[["item_id","content_type", "year", "genres"]], ["item_id"])

In [51]:
# df_train.withColumn("genres", f.split(f.col("genres"), ",")).select("genres.*").show(6)

In [52]:
# mean join
df_mean_user_purch=df_train.groupBy("user_id").mean("purchase")
df_mean_item_purch=df_train.groupBy("item_id").mean("purchase")

df_train=df_train.join(df_mean_user_purch, ["user_id"])
df_train=df_train.withColumnRenamed("avg(purchase)","mean_user_purch")
df_train=df_train.join(df_mean_item_purch, ["item_id"])
df_train=df_train.withColumnRenamed("avg(purchase)","mean_item_purch")

df_test=df_test.join(df_mean_user_purch, ["user_id"])
df_test=df_test.withColumnRenamed("avg(purchase)","mean_user_purch")
df_test=df_test.join(df_mean_item_purch, ["item_id"])
df_test=df_test.withColumnRenamed("avg(purchase)","mean_item_purch")

In [57]:
gbt = GBTClassifier(labelCol="purchase", featuresCol="features", maxIter=50, maxDepth=3) # 0.9003
lr = LogisticRegression(labelCol="purchase", featuresCol="features") # 0.889905372368

features_col = ["mean_user_purch", "mean_item_purch", "content_type"] # "year"
        # "count_user_purch", "count_item_purch", 
assembler = VectorAssembler(inputCols=features_col, outputCol="features")

from pyspark.ml.feature import StringIndexer
qualification_indexer = StringIndexer(inputCol="genres", outputCol="genres_index")

In [58]:
pipeline = Pipeline(stages=[assembler, gbt]) #  
model = pipeline.fit(df_train)
predictions = model.transform(df_test)

predictions.select("user_id","item_id", ith("probability", f.lit(1)).alias("purchase")).toPandas()\
        .sort_values(["user_id","item_id"]).to_csv('../lab03.csv', sep=',')

In [59]:
spark.stop()